# Generation of the distance matrice and its euclidean space transformation

This is the first part of the pipeline, where the data is preprocessed, distance matrice is generated and transformed into a coordinate matrice with a help of an MDS procedure, implemented in R. 

The data preprocessing includes labels (Y) and features (X) generation in a form of 1d and 2d numpy arrays respectively.

The distance matrice is obtained from the performances of each of the feature subsets on the cross-validation data set. The intrasubset distances are obtained based on the feature_importance parameter of Decision Tree classifier, sklearn implementation.  

# Importing all the necessary modules

In [ ]:
import pandas as pd
import gmql as gl
import random
import os
import sys
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import DistanceMetric
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import numpy as np
# matplotlib properties
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

#get_ipython().run_line_magic('matplotlib', 'inline')
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
matplotlib.rcParams['font.size'] = 22
matplotlib.rcParams['xtick.labelsize'] = 22
matplotlib.rcParams['ytick.labelsize'] = 22
matplotlib.rcParams['axes.labelsize'] = 30
matplotlib.rcParams['axes.titlesize'] = 30



# The file contains implementations of all the used functions with the desciptions

In [ ]:
from data_proc_dm_generation import *

# rpy2 module allows on usage of R inside of python

In [ ]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
sma = importr('smacof')

# Initialization of all the required parameters

In [ ]:
num_iters = 100 #number of iterations for finding the most optimal subset of features. 
#the subset is then subtracted from the list of available features and the operation is repeated 
subset_size = 5 #number of features to be chosen
top_k = 2000 # feature selectrion procedure, where top k features based on chi2 test are taken
tads = False #set to True if you want to work with tads set of genes

# Preprocessing data, generating features and labels for training and testing 

In [ ]:
if tads == True:
    path = "/home/nanni/Projects/ML/TAD_Trento/all_tcga_only_TADs.tsv"
else:
    path = "/home/nanni/Data/TCGA/all_tcga.tsv"

X_train,X_test,Y_train,Y_test = get_raw_data(path, top_k, tads)

## Obtaining subsets and their scores, building a distance matrice on its basis, applying MDS, checking the preservation of the nearest neighbors

In [ ]:
cluster_generator_wrapper_subsets(X_train, Y_train, num_iters, subset_size,top_k)
scores, ranks, inds = subset_scores_load(num_iters, subset_size, top_k, tads)
dist_final = get_distance_matrix(scores,inds,ranks,top_k)
R_mds = np.array(sma.torgerson(dist_final, p=dist_final.shape[0]-1))
pres = check_preservation_of_dims(R_mds.T, dist_final, subset_size)
print(pres)

# Saving coordinates matrice to the file

In [ ]:
temp_df = pd.DataFrame(R_mds)
temp_df.to_csv('./distance matrices/MDSes/test_dm_mds.csv',index=None,header=None)

# Second part of the pipeline - DL setup

## Imports 

In [ ]:
from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto(device_count = {'CPU': 1}, intra_op_parallelism_threads=7, inter_op_parallelism_threads=1)
session = tf.Session(config=config)
K.set_session(session)

import keras as k
import keras
from keras import applications
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.models import Model
from keras.layers import Input

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from keras.layers import (Lambda, MaxPooling1D, Flatten,
                          Dropout, Dense, Input)
from keras.models import Model
from keras.backend import floatx
from phcnn.layers import PhyloConv1D, euclidean_distances
from phcnn.build_train import *
from keras.utils.np_utils import to_categorical

%load_ext autoreload
%autoreload 2

## Making the results reproducible

In [1]:
seed = 7
np.random.seed(seed)

NameError: name 'np' is not defined

## Reshaping MDS represenation

In [ ]:
MDSmat = mds_reshape(R_mds, batch_size)

# Reshaping X and Y to the suitable form

In [ ]:
Y_input_test = keras.utils.to_categorical(Y_test, num_classes=None)
Y_input_train = keras.utils.to_categorical(Y_train, num_classes=None)

X_test_inp = X_test.reshape(X_test.shape[0],X_test.shape[1],-1)
X_train_inp = X_train.reshape(X_train.shape[0],X_train.shape[1],-1)

# Creating and training the model. Validation loss is printed

In [ ]:
model = create_model(X_train_inp, Y_input_train, MDSmat, nb_filters, nb_neighbors)
model_trained = train_model(model, X_train_inp, Y_input_train, batch_size, 7, 30, model_name, MDSmat, dense = False)   

## Evaluating the performance on the test set

In [ ]:
Y_pred = np.array(Y_input_test)
for i in range(0,Y_pred.shape[0]):
    Y_pred[i,:] = model.predict(x = [X_test_inp[[i],:,:]])#, MDSmat[1:2:,:,:,:]])

Y_pred_1d = to_1d_labels(Y_pred)
rep = classification_report(Y_test,Y_pred_1d)
print(rep)